## 주식 크롤링하기
(기존에 있던 자료 가져왔습니다)

In [2]:
#plotly 설명 : http://hamait.tistory.com/800

import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.offline as offline
import plotly.graph_objs as go

# jupyter notebook 에서 출력
offline.init_notebook_mode(connected=True)

print(plotly.__version__)
fig = go.Figure({
    "data": [Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": Layout(title="hello world")
})
offline.iplot(fig)

4.5.3


In [3]:
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import Scatter, Layout

# Create random data with numpy
import numpy as np

# jupyter notebook 에서 출력
offline.init_notebook_mode(connected=True)

N = 1000
random_x = np.random.randn(N)
random_y = np.random.randn(N)

# Create a trace
trace = go.Scatter(
    x = random_x,
    y = random_y,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
offline.iplot(data)


#출처: http://hamait.tistory.com/800 [HAMA 블로그]


In [17]:
#pip3 install pandas
#pip3 install plotly
#pip3 install lxml
#pip3 install BeautifulSoup4

import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

#print(code_df)

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
#print(code_df)

# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
#print(code_df.head())

# # 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# # 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df):
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}#'.format(code=code)
    url = url.replace(" ","")
    print("요청 URL = {}".format(url)) 
    return url

# # 신라젠의 일자데이터 url 가져오기 
item_name='AK홀딩스'
url = get_url(item_name, code_df)
# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 
# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 21):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() # 상위 5개 데이터 확인하기 
#print(df.head())

# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
    = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
print(df.head())

# #출처: http://excelsior-cjh.tistory.com/109 [EXCELSIOR]

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=006840#
          date  close  diff   open   high    low  volume
148 2020-03-09  23750  1550  25150  25150  23350   38061
58  2020-03-09  23750  1550  25150  25150  23350   38061
73  2020-03-09  23750  1550  25150  25150  23350   38061
88  2020-03-09  23750  1550  25150  25150  23350   38061
103 2020-03-09  23750  1550  25150  25150  23350   38061


In [33]:
#pip3 install pandas
#pip3 install plotly
#pip3 install lxml
#pip3 install BeautifulSoup4

import pandas as pd

def get_macd(df, short=12, long=26, t=9): 
    # 입력받은 값이 dataframe이라는 것을 정의해줌 
    df = pd.DataFrame(df) 
    # MACD 관련 수식 
    ma_12 = df.close.ewm(span=12).mean() 
    # 단기(12) EMA(지수이동평균) 
    ma_26 = df.close.ewm(span=26).mean() 
    # 장기(26) EMA 
    macd = ma_12 - ma_26 
    # MACD 
    macds = macd.ewm(span=9).mean() 
    # Signal 
    macdo = macd - macds 
    # Oscillator 
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna()
    return df

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

#print(code_df)

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
#print(code_df)

# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
#print(code_df.head())

# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df):
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}#'.format(code=code).replace(" ","")
    print("요청 URL = {}".format(url)) 
    return url

# 신라젠의 일자데이터 url 가져오기 
item_name='신라젠'
url = get_url(item_name, code_df)

# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 
# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 21):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() # 상위 5개 데이터 확인하기 
#print(df.head())

# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
    = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 
print(df.dtypes)

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
print(df.head())

df = get_macd(df)
print(df.head())

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600#
date      datetime64[ns]
close              int32
diff               int32
open               int32
high               int32
low                int32
volume             int32
dtype: object
          date  close  diff   open   high    low   volume
148 2020-03-09  11150   650  11550  11650  11100  1326762
58  2020-03-09  11150   650  11550  11650  11100  1326762
73  2020-03-09  11150   650  11550  11650  11100  1326762
88  2020-03-09  11150   650  11550  11650  11100  1326762
103 2020-03-09  11150   650  11550  11650  11100  1326762
          date  close  diff   open   high    low   volume  macd  macds  macdo
148 2020-03-09  11150   650  11550  11650  11100  1326762   0.0    0.0    0.0
58  2020-03-09  11150   650  11550  11650  11100  1326762   0.0    0.0    0.0
73  2020-03-09  11150   650  11550  11650  11100  1326762   0.0    0.0    0.0
88  2020-03-09  11150   650  11550  11650  11100  1326762   0.0    0.0    0.0
103 2020

In [34]:
from plotly import tools
import plotly.offline as offline 
import plotly.graph_objs as go

macd = go.Scatter( x=df.date, y=df['macd'], name="MACD")
signal = go.Scatter( x=df.date, y=df['macds'], name="Signal")
oscillator = go.Bar( x=df.date, y=df['macdo'], name="oscillator")
trade_volume = go.Bar( x=df.date, y=df['volume'], name="volume")
data = [macd, signal, oscillator]

# data = [celltrion] 
layout = go.Layout(title='{} MACD 그래프'.format(item_name)) 
fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True) 
for trace in data: 
    fig.append_trace(trace, 1,1) 

fig.append_trace(trade_volume, 2,1) # fig = go.Figure(data=data, layout=layout) 

offline.iplot(fig)

C:\Anaconda3\envs\tf_test\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

